<a href="https://colab.research.google.com/github/jzheng23/colab/blob/main/Screen_time_firebase_monitor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook provides the codes to check the progress of each participant, including two main functions:
1. Check the last update of screen events from each participant
2. Generate a list of emails (to be copied), to whom we should send a reminder email

# Preparation

## Mount Google Drive and set up file path

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Load the Google Drive path, depending who is running the notebook

In [3]:
#Jian
google_drive_data_path = '/content/drive/MyDrive/Problematic smartphone usage/Ambient display/Data'
google_drive_key_path = '/content/drive/MyDrive/Problematic smartphone usage/Ambient display/Key'

In [ ]:
#Subin
# google_drive_data_path = '/content/drive/MyDrive/UMD_research/Problematic_Smartphone_Usage'
# google_drive_key_path = '/content/drive/MyDrive/UMD_research/Problematic_Smartphone_Usage'

In [4]:
import pandas as pd
import numpy as np
import datetime
#Convert Unix timestamp to date/datetime in specified timezone
def convert_unix_to_date(unix_timestamp, timezone='America/New_York', return_date=True):
   """

   Parameters:
       unix_timestamp: Unix timestamp in milliseconds
       timezone: String of timezone (default 'America/New_York')
       return_date: If True returns date only, if False returns datetime
   """
   dt = pd.to_datetime(unix_timestamp, unit='ms', utc=True).tz_convert(timezone)
   return dt.date() if return_date else dt

## Google sheet module

In [5]:
from google.colab import auth
from google.auth import default
import gspread

# Authenticate
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [6]:
import pandas as pd

def get_sheet_as_df(spreadsheet_id):
    """
    Read Google Sheet and return as pandas DataFrame

    Args:
        spreadsheet_id (str): The ID from the Google Sheet URL

    Returns:
        pandas.DataFrame: Data from the sheet
    """
    try:
        # Open the spreadsheet
        spreadsheet = gc.open_by_key(spreadsheet_id)

        # Get first worksheet
        worksheet = spreadsheet.sheet1

        # Get all values including headers
        all_values = worksheet.get_all_values()

        # Convert to DataFrame
        df = pd.DataFrame(all_values[1:], columns=all_values[0])

        return df

    except Exception as e:
        print(f"Error accessing spreadsheet: {e}")
        return None



In [7]:
def write_df_to_sheet(df, spreadsheet_id):
    try:
        df_clean = df.copy()

        # Convert datetime columns
        date_cols = df_clean.select_dtypes(include=['datetime64[ns]', 'datetime64']).columns
        for col in date_cols:
            df_clean[col] = df_clean[col].dt.strftime('%Y-%m-%d')

        # Convert numeric columns to handle inf values first
        df_clean = df_clean.replace([np.inf, -np.inf], 'Infinity')

        # Convert all columns to string explicitly
        for col in df_clean.columns:
            df_clean[col] = df_clean[col].astype(str)

        df_clean = df_clean.fillna('')

        spreadsheet = gc.open_by_key(spreadsheet_id)
        worksheet = spreadsheet.sheet1

        values = [df_clean.columns.values.tolist()] + df_clean.values.tolist()
        worksheet.clear()
        worksheet.update(values=values, range_name='A1')

        return True

    except Exception as e:
        print(f"Error writing to spreadsheet: {e}")
        return False

# Firebase database

## Set up link

In [8]:
import firebase_admin
from firebase_admin import credentials, db
import csv
import pandas as pd

In [9]:
# Don't re-run this cell unless the kernel has been restarted
cred = credentials.Certificate(google_drive_key_path+'/timer-42ad2-firebase-adminsdk-4r7oj-2c373565f2.json')
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://timer-42ad2-default-rtdb.firebaseio.com'
})

In [10]:
#Convert Unix timestamp to date/datetime in specified timezone
def convert_unix_to_date(unix_timestamp, timezone='America/New_York', return_date=True):
   """

   Parameters:
       unix_timestamp: Unix timestamp in milliseconds
       timezone: String of timezone (default 'America/New_York')
       return_date: If True returns date only, if False returns datetime
   """
   dt = pd.to_datetime(unix_timestamp, unit='ms', utc=True).tz_convert(timezone)
   return dt.date() if return_date else dt

## Response data

In [11]:
def process_responses(responses, pid, host, survey_id):
    responses_data = []
    response_dict = {
        'pid': pid,
        'host': host,
        'surveyID': survey_id
    }

    if isinstance(responses, dict):
        for question_id, answer in responses.items():
            response_dict[f'q_{question_id}'] = answer
    elif isinstance(responses, list):
        for question_id, answer in enumerate(responses):
            if answer is not None:
                response_dict[f'q_{question_id}'] = answer

    responses_data.append(response_dict)
    return responses_data

def get_timestamp(data):
    if isinstance(data, dict):
        return data.get('a') or data.get('timestamp')
    return None

def get_responses(data):
    if isinstance(data, dict):
        return data.get('b') or data.get('responses')
    elif isinstance(data, list):
        return data
    return None

def firebase_to_csv_log_Screen(ref_path, output_file):
    ref = db.reference(ref_path)
    data = ref.get()
    transformed_data = []

    for pid, pid_data in data.items():
        pid = pid.lower()

        # Process each host under the pid
        for host, events in pid_data.items():
            if isinstance(events, list):
                for index, event in enumerate(events):
                    if event is not None and isinstance(event, dict):
                        row = {
                            'pid': pid,
                            'host': host,
                            'eventLabel': index,
                            'startTime': event.get('startTime'),
                            'endTime': event.get('endTime')
                        }
                        transformed_data.append(row)

    df = pd.DataFrame(transformed_data)
    if len(transformed_data) > 0:
        df = df[['pid', 'host', 'eventLabel', 'startTime', 'endTime']]
    df.to_csv(output_file, index=False)

def responses_to_csvs(meta_file, responses_file):
    ref = db.reference('/responses')
    data = ref.get()

    meta_data = []
    responses_data = []

    for pid, pid_data in data.items():
        pid = pid.lower()

        # Process each host under pid
        for host, host_data in pid_data.items():
            # If host_data is a list, enumerate through it
            if isinstance(host_data, list):
                for survey_id, survey_data in enumerate(host_data):
                    if survey_data is not None:
                        timestamp = get_timestamp(survey_data)
                        if timestamp:
                            meta_data.append({
                                'pid': pid,
                                'host': host,
                                'surveyID': str(survey_id),
                                'timestamp': timestamp
                            })

                            responses = get_responses(survey_data)
                            if responses:
                                responses_data.extend(process_responses(responses, pid, host, str(survey_id)))

    # Create and save metadata DataFrame
    meta_df = pd.DataFrame(meta_data)
    if len(meta_data) > 0:
        meta_df = meta_df[['pid', 'host', 'surveyID', 'timestamp']]
    meta_df.to_csv(meta_file, index=False)

    # Create responses DataFrame in wide format
    responses_df = pd.DataFrame(responses_data)

    if len(responses_data) > 0:
        first_cols = ['pid', 'host', 'surveyID']
        q_cols = [col for col in responses_df.columns if col.startswith('q_')]
        q_cols.sort(key=lambda x: int(x.split('_')[1]))
        responses_df = responses_df[first_cols + q_cols]

    responses_df.to_csv(responses_file, index=False)

In [12]:
responses_to_csvs('survey_meta.csv', 'survey_responses.csv')
survey_meta_data = pd.read_csv('/content/survey_meta.csv')
print("The shape of the survey_meta_data is " + str(survey_meta_data.shape))
survey_meta_data.to_csv(google_drive_data_path + '/survey_meta.csv', index=False)
survey_responses_data = pd.read_csv('/content/survey_responses.csv')
print("The shape of the survey_responses_data is " + str(survey_responses_data.shape))
survey_responses_data.to_csv(google_drive_data_path + '/survey_responses.csv', index=False)

The shape of the survey_meta_data is (41, 4)
The shape of the survey_responses_data is (41, 26)


## Screen events

In [13]:
def firebase_to_csv_log_Screen(ref_path, output_file):
    ref = db.reference(ref_path)
    data = ref.get()
    transformed_data = []

    for pid, pid_data in data.items():
        pid = pid.lower()

        if not isinstance(pid_data, dict):
            continue

        for host, events in pid_data.items():
            if isinstance(events, list):
                # Handle list structure
                for index, event in enumerate(events):
                    if event is None:
                        continue
                    if isinstance(event, dict):
                        row = {
                            'pid': pid,
                            'host': host,
                            'eventLabel': index,
                            'startTime': event.get('startTime'),
                            'endTime': event.get('endTime')
                        }
                        transformed_data.append(row)
            elif isinstance(events, dict):
                # Handle dictionary structure
                for event_label, event_data in events.items():
                    if isinstance(event_data, dict):
                        row = {
                            'pid': pid,
                            'host': host,
                            'eventLabel': event_label,
                            'startTime': event_data.get('startTime'),
                            'endTime': event_data.get('endTime')
                        }
                        transformed_data.append(row)

    df = pd.DataFrame(transformed_data)
    if len(transformed_data) > 0:
        df = df[['pid', 'host', 'eventLabel', 'startTime', 'endTime']]
    df.to_csv(output_file, index=False)

In [14]:
firebase_to_csv_log_Screen('/screen_events','screen_events.csv')
screen_events_data = pd.read_csv('/content/screen_events.csv')
print("The shape of the screen_events_data is " + str(screen_events_data.shape))
screen_events_data.to_csv(google_drive_data_path + '/screen_events.csv', index=False)

The shape of the screen_events_data is (9319, 5)


In [15]:
# prompt: count how many row under each unique pid in the screen_events_data

# row_counts = screen_events_data.groupby('pid').size().reset_index(name='row_count')
# row_counts

## Device info

In [16]:
# this function works for device and timezone infor
def firebase_to_csv(ref_path, output_file):
    # Get reference to device_info
    ref = db.reference(ref_path)
    data = ref.get()

    transformed_data = []

    # Transform the nested structure
    for pid, pid_data in data.items():
        pid = pid.lower()

        if isinstance(pid_data, str):
            # Case where timezone is directly under pid
            row = {
                'pid': pid,
                'host': pd.NA,  # or None
                'timezone': pid_data
            }
            transformed_data.append(row)
        else:
            # Case where pid has host-timezone pairs
            for host, timezone in pid_data.items():
                row = {
                    'pid': pid,
                    'host': host,
                    'timezone': timezone
                }
                transformed_data.append(row)

    # Convert to DataFrame
    df = pd.DataFrame(transformed_data)

    # Ensure pid and host are the first columns
    cols = ['pid', 'host'] + [col for col in df.columns if col not in ['pid', 'host']]
    df = df[cols]

    # Save to CSV
    df.to_csv(output_file, index=False)

In [17]:
firebase_to_csv('/device_info','device_info.csv')
device_data = pd.read_csv('/content/device_info.csv')

# First let's convert the string representation of dictionary to actual dictionary
device_data0 = device_data.copy()
device_data0['timezone'] = device_data0['timezone'].apply(eval)

# Now extract each field into its own column
device_data0['Base'] = device_data0['timezone'].apply(lambda x: x.get('Base'))
device_data0['Brand'] = device_data0['timezone'].apply(lambda x: x.get('Brand'))
device_data0['Model'] = device_data0['timezone'].apply(lambda x: x.get('Model'))

# If you want to drop the original timezone column, you can use:
device_data = device_data0.drop('timezone', axis=1)

print("The shape of the device_data is " + str(device_data.shape))
device_data.to_csv(google_drive_data_path + '/device_info.csv', index=False)

The shape of the device_data is (38, 5)


## Timezone info

In [18]:
firebase_to_csv('/timezones','timezones.csv')
timezones_data = pd.read_csv('/content/timezones.csv')
print("The shape of the timezones_data is " + str(timezones_data.shape))
timezones_data.to_csv(google_drive_data_path + '/timezones.csv', index=False)

The shape of the timezones_data is (39, 3)


## User states

This data was introduced in an update of the app, so it only exist in some of the participants' data.

In [19]:
firebase_to_csv('/user_states','user_states.csv')
user_states = pd.read_csv('/content/user_states.csv')

In [20]:
# First convert the string representation of dictionary to actual dictionary
user_states['timezone'] = user_states['timezone'].apply(eval)

# Extract each field into its own column
user_states['day_count'] = user_states['timezone'].apply(lambda x: x.get('day_count'))
user_states['survey1_completed'] = user_states['timezone'].apply(lambda x: x.get('survey1_completed'))
user_states['survey2_completed'] = user_states['timezone'].apply(lambda x: x.get('survey2_completed'))
user_states['survey3_completed'] = user_states['timezone'].apply(lambda x: x.get('survey3_completed'))
user_states['survey4_completed'] = user_states['timezone'].apply(lambda x: x.get('survey4_completed'))
user_states['tutorial_completed'] = user_states['timezone'].apply(lambda x: x.get('tutorial_completed'))

# Drop the original timezone column
user_states = user_states.drop('timezone', axis=1)

# Display the result
print("The shape of the user_states is " + str(user_states.shape))
user_states.to_csv(google_drive_data_path + '/user_states.csv', index=False)


The shape of the user_states is (37, 8)


## Setting changes

Data of setting changes will come in the third week. Don't run this chuck of code early than that, it will return just error.



In [ ]:
def firebase_to_csv_log_setting (ref_path, output_file):
    ref = db.reference(ref_path)
    data = ref.get()
    transformed_data = []

    # dictionary structure
    for pid, changes in data.items():  # 'pid' = key
        if isinstance(changes, list):
            for index, change in enumerate(changes, start=0):  #event_label_start:1
                if change is None:
                    continue
                if isinstance(change, dict):
                    row = {'pid': pid,
                        'eventLabel': index,
                        'newValue' : change.get('newValue'),
                        'setting': change.get('setting'),
                        'timestamp': change.get('timestamp')}
                    transformed_data.append(row)

    df = pd.DataFrame(transformed_data)
    df = df[['pid', 'newValue', 'setting', 'timestamp']]
    df.to_csv(output_file, index=False)

In [ ]:
firebase_to_csv_log_setting('/settings_change_logs','settings_change_logs.csv')
settings_change_logs_data = pd.read_csv('/content/settings_change_logs.csv')
print("The shape of the settings_change_logs_data is " + str(settings_change_logs_data.shape))
settings_change_logs_data.to_csv(google_drive_data_path + '/settings_change_logs.csv', index=False)

AttributeError: 'NoneType' object has no attribute 'items'

## Ringer

Data of ringer changes will come in the third week. Don't run this chuck of code early than that, it will return just error.

In [ ]:
def firebase_to_csv_log_mode(ref_path, output_file):
    ref = db.reference(ref_path)
    data = ref.get()
    transformed_data = []

    for pid, middle_levels in data.items():
        pid = pid.lower()
        for middle_key, sub_changes in middle_levels.items():
            for index, event in enumerate(sub_changes):
                if isinstance(event, dict):
                    row = {'pid': pid,'host':middle_key,'label': str(index),
                        'mode': event.get('mode'),
                        'timestamp': event.get('timestamp')}
                    transformed_data.append(row)

    df = pd.DataFrame(transformed_data)
    df = df[['pid', 'host', 'label', 'mode', 'timestamp']]
    df.to_csv(output_file, index=False)

In [ ]:
firebase_to_csv_log_mode('/ringer_mode_events','ringer_mode_events.csv')
ringer_mode_events_data = pd.read_csv('/content/ringer_mode_events.csv')
print("The shape of the ringer_mode_events_data is " + str(ringer_mode_events_data.shape))
ringer_mode_events_data.to_csv(google_drive_data_path + '/ringer_mode_events.csv', index=False)

## Counters

We don't actually need top analyze the counter data

In [ ]:
# def firebase_to_csv_counters(ref_path, output_file):
#     ref = db.reference(ref_path)
#     data = ref.get()
#     transformed_data = []
#     for pid, value in data.items():
#         pid = pid.lower()
#         # Case 1: only numeric
#         if isinstance(value, int):
#             transformed_data.append({'pid': pid, 'screen_event_count': value})

#         # Case 2: dictionary => selecting only value
#         elif isinstance(value, dict):
#             numeric_values = [v for v in value.values() if isinstance(v, int)]
#             if numeric_values:
#                 total = sum(numeric_values)
#                 transformed_data.append({'pid': pid, 'screen_event_count': total})

#     df = pd.DataFrame(transformed_data)
#     df = df[['pid', 'screen_event_count']]
#     df.to_csv(output_file, index=False)


# firebase_to_csv_counters('/ringer_event_counters','ringer_event_counters.csv')
# ringer_event_counters_data0 = pd.read_csv('/content/ringer_event_counters.csv')
# print(ringer_event_counters_data0.head())

# firebase_to_csv_counters('/screen_event_counters','screen_event_counters.csv')
# screen_event_counters_data0 = pd.read_csv('/content/screen_event_counters.csv')
# print(screen_event_counters_data0.head())

# firebase_to_csv_counters('/settings_change_counters','settings_change_counters.csv')
# settings_change_counters_data0 = pd.read_csv('/content/settings_change_counters.csv')
# print(settings_change_counters_data0.head())

# Paricipant management

## Progress check

Which day is each participant on, the state of each survey and the tutorial: unavailable, available, or completed.

(Wait about 10 seconds) Read in the meta data

In [64]:
# read in the data
import pytz
from datetime import datetime

survey_data = pd.read_csv(google_drive_data_path + '/survey_meta.csv')
survey_data['submitDate'] = pd.to_datetime(survey_data['timestamp'].map(convert_unix_to_date))

# Pivot the dataframe to create survey columns
survey_meta_pivoted = survey_data.pivot(index=['pid','host'], columns='surveyID', values='submitDate')

# Rename the columns
survey_meta_pivoted.columns = ['survey' + str(col) for col in survey_meta_pivoted.columns]

# Reset the index to make pid a column again
survey_meta_pivoted = survey_meta_pivoted.reset_index()


# Get today's date once
ny_tz = pytz.timezone('America/New_York')
today = pd.Timestamp.now(tz=ny_tz).normalize().date()

# Fixed date operations
# survey_meta_pivoted['survey1'] = pd.to_datetime(survey_meta_pivoted['survey1']).dt.date

cols_to_convert = survey_meta_pivoted.filter(like='survey').columns
for col in cols_to_convert:
    survey_meta_pivoted[col] = pd.to_datetime(survey_meta_pivoted[col]).dt.date

survey_meta_pivoted['real_day_count'] = np.array([today - d for d in survey_meta_pivoted['survey1']], dtype='timedelta64[D]').astype(int) + 1

# if 'survey2' in survey_meta_pivoted.columns:
#     survey_meta_pivoted['survey2'] = pd.to_datetime(survey_meta_pivoted['survey2']).dt.date

In [65]:
# First check which columns are missing
required_columns = ['survey2', 'survey3', 'survey4']
existing_columns = survey_meta_pivoted.columns.tolist()
missing_columns = [col for col in required_columns if col not in existing_columns]

# Create copy of dataframe and add missing columns with NA
df = survey_meta_pivoted.copy()
for col in missing_columns:
   df[col] = pd.NA

# Get current date in NY timezone as pandas Timestamp
today = pd.Timestamp.now(tz=ny_tz)


# Calculate days_since_s1
df['days_since_s1'] = df['real_day_count']

# Create app_day_count column
df['app_day_count'] = df['days_since_s1']

In [66]:
df.dtypes

,0
pid,object
host,int64
survey1,object
survey2,object
real_day_count,int64
survey3,object
survey4,object
days_since_s1,int64
app_day_count,int64


In [62]:
# Handle cases where survey2 is NA
df.loc[pd.isna(df['survey2']), 'app_day_count'] = df.loc[pd.isna(df['survey2']), 'days_since_s1'].clip(upper=8)

# Handle survey2 cases when survey3 is NA
df['survey2'] = pd.to_datetime(df['survey2']).dt.tz_localize(ny_tz)
mask_survey2 = pd.notna(df['survey2'])

# Handle survey3 cases when survey4 is NA
df['survey3'] = pd.to_datetime(df['survey3']).dt.tz_localize(ny_tz)
mask_survey3 = pd.notna(df['survey3'])

df['survey4'] = pd.to_datetime(df['survey4']).dt.tz_localize(ny_tz)

df['delay1'] = 0
df.loc[mask_survey2, 'delay1'] = ((df.loc[mask_survey2, 'survey2']
                                 - df.loc[mask_survey2, 'survey1']).dt.days - 7).clip(lower=0)

mask_survey2_no3 = mask_survey2 & pd.isna(df['survey3'])
df.loc[mask_survey2_no3, 'app_day_count'] = (df.loc[mask_survey2_no3, 'days_since_s1']
                                           - df.loc[mask_survey2_no3, 'delay1']).clip(upper=15)

df['delay2_threshold'] = 7
df.loc[mask_survey3 & ((df['survey2'] - df['survey1']).dt.days == 6), 'delay2_threshold'] = 8

df['delay2'] = 0
df.loc[mask_survey3, 'delay2'] = ((df.loc[mask_survey3, 'survey3'] - df.loc[mask_survey3, 'survey2']).dt.days
                                - df.loc[mask_survey3, 'delay2_threshold']).clip(lower=0)

mask_survey3_no4 = mask_survey3 & pd.isna(df['survey4'])
df.loc[mask_survey3_no4, 'app_day_count'] = (df.loc[mask_survey3_no4, 'days_since_s1']
                                           - df.loc[mask_survey3_no4, 'delay1']
                                           - df.loc[mask_survey3_no4, 'delay2']).clip(upper=22)

# Handle survey4 cases
df.loc[pd.notna(df['survey4']), 'app_day_count'] = 22
survey_meta_pivoted = df[['pid', 'host', 'survey1', 'survey2', 'survey3', 'survey4', 'real_day_count', 'app_day_count']]
survey_meta_pivoted = (survey_meta_pivoted
   .assign(pid=lambda x: x['pid'].str.upper())
   .rename(columns={'pid': 'PID'}))


<ipython-input-62-9cf1f508744b>:15: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  df.loc[mask_survey2, 'delay1'] = ((df.loc[mask_survey2, 'survey2']


TypeError: unsupported operand type(s) for -: 'Timestamp' and 'datetime.date'

Read the email info and merge

Before running the cell below, make sure the Participants.xlsx is up to date.

In [24]:
participant_data = pd.read_excel(google_drive_data_path + '/Participants.xlsx')

participant_manager = pd.merge(survey_meta_pivoted, participant_data, on='PID', how='right')

# Sort by survey1
participant_manager = participant_manager.sort_values(by='survey1')

# Drop the specified columns
participant_manager = participant_manager.drop(columns=['host', 'real_day_count'], errors='ignore')

# Rename the 'Date' column to 'Date_invited' in the participant_manager DataFrame.
participant_manager = participant_manager.rename(columns={'Date': 'date_invited'})

participant_manager.drop(columns=['nick name'], errors='ignore').to_csv(google_drive_data_path + '/survey_meta_pivot.csv', index=False)
# Read and process screen data in one chain
latest_times = (pd.read_csv(google_drive_data_path + '/screen_events.csv')
   .groupby('pid')['endTime']
   .max()
   .reset_index()
   .assign(lastUpdate=lambda x: x['endTime'].map(lambda y: convert_unix_to_date(y, return_date=False)))
   .drop(columns=['endTime'])
   .assign(PID=lambda x: x['pid'].str.upper())
   .drop(columns=['pid']))

# Read and process timezone data in one chain
timezone_data = (pd.read_csv(google_drive_data_path + '/timezones.csv')
   .assign(PID=lambda x: x['pid'].str.upper())
   .drop(columns=['pid'])
   [['PID', 'timezone']])

# Combine all merges and data cleaning in one chain
participant_manager = (participant_manager
   .merge(latest_times, on='PID', how='left')
   .merge(timezone_data, on='PID', how='left')
   .assign(lastUpdate=lambda x: x['lastUpdate'].astype(str).replace('NaT', ''))
   .fillna(''))

Update lastUpdate with Timezone

In [25]:
import pandas as pd
from datetime import datetime
import pytz

def convert_to_local_time(row):
    # Skip rows where either lastUpdate or timezone is missing
    if pd.isna(row['lastUpdate']) or pd.isna(row['timezone']):
        return pd.NaT

    try:
        # Parse the timestamp
        dt = pd.to_datetime(row['lastUpdate'])
        if pd.isna(dt):
            return pd.NaT

        # Get source timezone (EST/EDT)
        source_tz = pytz.timezone('America/New_York')

        # Localize the datetime
        dt_source = source_tz.localize(dt.tz_localize(None))

        # Convert to target timezone
        target_tz = pytz.timezone(row['timezone'])
        dt_target = dt_source.astimezone(target_tz)

        return dt_target

    except (ValueError, AttributeError):
        return pd.NaT

# Apply the conversion
participant_manager['local_time'] = participant_manager.apply(convert_to_local_time, axis=1)

# Replace "NaT" with empty strings in the 'lastUpdate' column
participant_manager['local_time'] = participant_manager['local_time'].astype(str).replace('NaT', '')
def calculate_time_lapse(last_update):
    if pd.isna(last_update):
        return None

    # Get current time in EST (since lastUpdate is in EST)
    est = pytz.timezone('America/New_York')
    now = datetime.now(est)

    try:
        # Parse lastUpdate with timezone info (-05:00 is already in the data)
        last_update = pd.to_datetime(last_update, utc=True)
        last_update = last_update.astimezone(est)

        # Calculate time difference
        time_diff = now - last_update

        # Convert to hours and minutes
        total_minutes = time_diff.total_seconds() / 60
        hours = int(total_minutes // 60)
        minutes = int(total_minutes % 60)

        # Format as HH:MM
        return f"{hours:02d}:{minutes:02d}"
    except:
        return None

# Calculate time lapse for each row
participant_manager['time_lapse'] = participant_manager['lastUpdate'].apply(calculate_time_lapse)

# prompt: Rename "nick name":"nick_name", "timezone":"time_zone", "local_time":"last_update_local","time_lapse":"lapse_last_update"

# Rename the columns
participant_manager = participant_manager.rename(columns={
    "survey1": "start_date",
    "nick name": "nick_name",
    "timezone": "time_zone",
    "local_time": "last_update_local",
    "time_lapse": "lapse_last_update"
})

print(participant_manager.columns)


Index(['PID', 'start_date', 'survey2', 'survey3', 'survey4', 'app_day_count',
       'Email', 'nick_name', 'date_invited', 'lastUpdate', 'time_zone',
       'last_update_local', 'lapse_last_update'],
      dtype='object')


In [26]:
# prompt: reorder the columns: PID, Email, nick_name, start_date, app_day_count, last_update_local, lapse_last_update

# Reorder the columns
new_column_order = ['PID', 'Email', 'nick_name', 'date_invited', 'start_date', 'survey2', 'survey3','survey4','app_day_count', 'last_update_local', 'lapse_last_update']
participant_manager = participant_manager[new_column_order]

Creating a status

In [27]:
participant_manager_ID = '1RxxvaVzf15X3H0ak4UTDr4pjrBRjoCCiOW1smkTTJLs'
saved_status = get_sheet_as_df(participant_manager_ID)
# saved_status = saved_status.drop(columns=['PID', 'nick_name', 'start_date', 'app_day_count', 'last_update_local', 'lapse_last_update'], errors='ignore')

import pandas as pd
from datetime import datetime, time

# Read the CSV
df = participant_manager.copy()

# Identify duplicate columns
duplicate_cols = [col for col in df.columns if col in saved_status.columns and col != 'Email']

In [28]:
# Drop the duplicate columns from the right DataFrame before merging
saved_status = saved_status.drop(columns=duplicate_cols, errors='ignore')
saved_status = saved_status.drop(columns=['daily_unlocks', 'daily_screen_hours', 'day_count','day_missing','note'], errors='ignore')

df = df.merge(saved_status, on='Email', how='left')

# Convert 'status' column to numeric, handling errors
df['status'] = pd.to_numeric(df['status'], errors='coerce')

# Initialize status column with 1 ONLY where status is NaN or < 4
mask_initial = df['status'].isna() | (df['status'] < 8)
df.loc[mask_initial, 'status'] = 1


In [29]:
# Get all columns that are numbers as strings
numeric_cols = [col for col in df.columns if str(col).isdigit()]

# Replace NaT in datetime columns
datetime_cols = ['date_invited', 'start_date', 'survey2', 'survey3', 'survey4']
df[datetime_cols] = df[datetime_cols].astype(object).where(df[datetime_cols].notnull(), '')

# Replace nulls in object columns
object_cols = df.select_dtypes(include=['object']).columns
df[object_cols] = df[object_cols].fillna('')

# Replace string 'nan' in numeric columns
df[numeric_cols] = df[numeric_cols].replace('nan', '')

# For the int64 column (status)
df['status'] = df['status'].fillna('')

<ipython-input-29-547b309ecef8>:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[object_cols] = df[object_cols].fillna('')


In [30]:
df.to_csv(google_drive_data_path + '/participant_manager_temp.csv', index=False)

In [31]:
df.head()

,PID,Email,nick_name,date_invited,start_date,survey2,survey3,survey4,app_day_count,last_update_local,lapse_last_update,status,1210,1209,1208,1207,1206,1205,Source
0,ST011,mappscarl@gmail.com,Solo,2024-12-04,2024-12-03 19:00:00-05:00,,,,8.0,2024-12-10 15:15:32.156000-06:00,02:56,1,survey awaiting,working well,data paused,working well,data paused,,Janel Rana
1,ST012,djsboca@aol.com,David,2024-12-04,2024-12-03 19:00:00-05:00,2024-12-10 00:00:00-05:00,,,8.0,2024-12-10 18:23:48.718000-05:00,00:47,1,working well,data paused,data paused,data paused,,data paused,Janel Rana
2,ST013,santanana2000@yahoo.com,dan,2024-12-04,2024-12-03 19:00:00-05:00,2024-12-10 00:00:00-05:00,,,8.0,2024-12-10 18:08:47.993000-06:00,00:02,1,working well,working well,working well,working well,,,Janel Rana
3,ST014,sadiesantacruz@gmail.com,Sadie,2024-12-04,2024-12-03 19:00:00-05:00,,,,8.0,2024-12-10 18:08:59.581000-06:00,00:02,1,survey awaiting,working well,working well,working well,,,Janel Rana
4,ST015,jimoreno2055@gmail.com,Juan,2024-12-04,2024-12-03 19:00:00-05:00,,,,8.0,2024-12-10 11:22:23.778000-03:00,09:49,1,survey awaiting,data paused,working well,working well,data paused,,Janel Rana


In [32]:
df = pd.read_csv(google_drive_data_path + '/participant_manager_temp.csv')
# Convert lapse_last_update to datetime.time objects where possible
def convert_time(x):
    try:
        if pd.notna(x):
            hours, minutes = map(int, x.split(':'))
            return time(hours, minutes)
    except:
        return pd.NaT

df['lapse_time'] = df['lapse_last_update'].apply(convert_time)

# Set status based on conditions, but only where status < 4 or NaN
cutoff_time = time(12, 0)
mask_eligible = (df['status'] < 8) | (df['status'].isna())

# New conditions (checking for empty strings)
# If both start_date and last_update_local are empty strings
mask_both_empty = (df['start_date'] == '') & (df['last_update_local'] == '')
df.loc[mask_eligible & mask_both_empty, 'status'] = 1

# If start_date is empty string but last_update_local is not
mask_only_update = (df['start_date'] == '') & (df['last_update_local'] != '')
df.loc[mask_eligible & mask_only_update, 'status'] = 4

# If start_date is not empty string but last_update_local is empty string
mask_only_start = (df['start_date'] != '') & (df['last_update_local'] == '')
df.loc[mask_eligible & mask_only_start, 'status'] = 3

# Original conditions
# 1. If lapse_last_update < 12:00, status = "started"
mask_started = df['lapse_time'].apply(lambda x: pd.notna(x) and isinstance(x, time) and x < cutoff_time)
df.loc[mask_eligible & mask_started, 'status'] = 2

# 2. If start_date is not empty but lapse_last_update is empty or ≥ 12:00
mask_tech_issue = (df['start_date'] != '') & (~mask_started)
df.loc[mask_eligible & mask_tech_issue, 'status'] = 3

# New survey conditions for status 4
mask_survey2 = (df['app_day_count'].isin([7, 8])) & (df['survey2'].isna() | (df['survey2'] == ''))
mask_survey3 = (df['app_day_count'].isin([14, 15])) & (df['survey3'].isna() | (df['survey3'] == ''))
mask_survey4 = (df['app_day_count'].isin([21, 22])) & (df['survey4'].isna() | (df['survey4'] == ''))

# Combine all survey conditions
mask_surveys = mask_survey2 | mask_survey3 | mask_survey4

# Add the new condition to set status = 4
df.loc[mask_eligible & mask_surveys, 'status'] = 4


In [33]:
# Drop the temporary lapse_time column
df = df.drop('lapse_time', axis=1)
import datetime
from datetime import timezone
import pytz

# Get today's date in New York time
ny_tz = pytz.timezone('America/New_York')
today = datetime.datetime.now(ny_tz).strftime('%m%d')
# Drop today's column if it already exists
if int(today) in df.columns:
    df = df.drop(columns=[int(today)])

# Create status-note mapping dictionary from the CSV data
status_mapping = {
    0: 'invitation expired',
    1: 'invited',
    2: 'working well',
    3: 'data paused',
    4: 'survey awaiting',
    8: 'dropped out',
    9: 'completed'
}

# Create a new column with today's date and initialize it with NaN
df[today] = float('nan')

# Update the new column using the status mapping
df.loc[df[today].isna(), today] = df.loc[df[today].isna(), 'status'].map(status_mapping)

# Get the fixed columns we want first (up to status)
fixed_cols = ['PID', 'Email', 'nick_name', 'date_invited', 'start_date', 'survey2',
       'survey3', 'survey4', 'app_day_count', 'last_update_local', 'lapse_last_update', 'status']

# Add today's column
fixed_cols.append(today)

# Get the remaining date columns in descending order
# Use the original column types from df.columns
date_cols = [col for col in df.columns if str(col).isdigit() and col != today]
date_cols = sorted(date_cols, key=lambda x: int(str(x)), reverse=True)

# Combine all columns in desired order
new_order = fixed_cols + date_cols

# Reorder the dataframe
df1 = df[new_order].copy()

sign_up_data = pd.read_excel(google_drive_data_path + '/signup_survey_data_processed_combined.xlsx')

# prompt: Drop 'Source' in df1 if there is such a column. Then merge sign_update[['Email','Source']] to df1.

if 'Source' in df1.columns:
  df1 = df1.drop('Source', axis=1)

df1 = pd.merge(df1, sign_up_data[['Email', 'Source']], on='Email', how='left')

<ipython-input-33-f56f0642d3d3>:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['survey awaiting' 'working well' 'working well' 'survey awaiting'
 'survey awaiting' 'working well' 'survey awaiting' 'dropped out'
 'survey awaiting' 'survey awaiting' 'survey awaiting' 'survey awaiting'
 'dropped out' 'survey awaiting' 'survey awaiting' 'survey awaiting'
 'survey awaiting' 'survey awaiting' 'survey awaiting' 'working well'
 'working well' 'working well' 'working well' 'working well'
 'working well' 'working well' 'working well' 'working well'
 'working well' 'working well' 'dropped out' 'working well' 'working well'
 'working well' 'working well' 'working well' 'working well'
 'working well' 'data paused' 'data paused' 'data paused' 'data paused'
 'data paused' 'data paused' 'data paused' 'data paused' 'data paused'
 'data paused' 'data paused']' has dtype incompatible with float64, please explicitly cas

In [34]:
screen_data = pd.read_csv(google_drive_data_path + '/screen_events.csv')
# Calculate duration in milliseconds by simple subtraction since values are already timestamps
screen_data['duration_ms'] = screen_data['endTime'] - screen_data['startTime']
# prompt: filter out where duration_ms <= 0 in screen_data
print(screen_data.shape)
screen_data = screen_data[screen_data['duration_ms'] > 0]
print(screen_data.shape)
timezones = pd.read_csv(google_drive_data_path + '/timezones.csv')
# First merge the timezone information with screen_data
screen_data = screen_data.merge(timezones[['pid', 'timezone']], on='pid', how='left')

# Calculate statDate using the timezone specific to each row
screen_data['startDate'] = screen_data.apply(
    lambda row: convert_unix_to_date(
        row['startTime'],
        timezone=row['timezone'],
        return_date=True
    ),
    axis=1
)
# Calculate the metrics grouped by pid and statDate
summary = screen_data.groupby(['pid', 'startDate']).agg({
    'eventLabel': 'count',  # Count of events
    'duration_ms': lambda x: sum(x) / (1000 * 60 * 60)  # Convert milliseconds to hours
}).reset_index()

# Rename columns for clarity
summary.columns = ['pid', 'startDate', 'event_count', 'duration_hours']

summary.to_csv(google_drive_data_path + '/screen_event_summary.csv', index=False)
# Sort by pid and date, then drop the first date for each pid
filtered_summary = summary.sort_values(['pid', 'startDate']).groupby('pid').apply(
    lambda x: x.iloc[1:] if len(x) > 1 else pd.DataFrame()
).reset_index(drop=True)
# Calculate the averages and add date count + 1
avg_metrics = filtered_summary.groupby('pid').agg({
    'event_count': 'mean',
    'duration_hours': 'mean',
    'startDate': lambda x: len(x.unique()) + 1  # count of remaining dates + 1
}).reset_index()

# Rename columns to indicate these are averages
avg_metrics.columns = ['PID', 'daily_unlocks', 'daily_screen_hours', 'day_count']

avg_metrics['PID'] = avg_metrics['PID'].str.upper()
# prompt: merge avg_metrics to df1 by PID, how = left

df1 = pd.merge(df1, avg_metrics, on='PID', how='left')


# Round daily_unlocks and daily_screen_hours to one decimal place
df1['daily_unlocks'] = df1['daily_unlocks'].round(1)
df1['daily_screen_hours'] = df1['daily_screen_hours'].round(1)



(9319, 6)
(8813, 6)


<ipython-input-34-6db3a76a14ee>:32: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_summary = summary.sort_values(['pid', 'startDate']).groupby('pid').apply(


In [35]:
df2 = df1.copy()

df2['app_day_count'] = pd.to_numeric(df2['app_day_count'], errors='coerce').fillna(0).astype(int)

df2['day_missing'] = df2['app_day_count'] - df2['day_count']
def create_note(row):

    # Check status=5 condition first
    if row['status'] == 5:
        return 'dropped out'

    # Check for missing values first
    if (pd.isna(row['app_day_count']) or pd.isna(row['status']) or
        pd.isna(row['daily_unlocks']) or pd.isna(row['daily_screen_hours']) or
        pd.isna(row['day_count']) or pd.isna(row['day_missing'])):
        return 'TBD'

    # Check app_day_count <= 3
    if row['app_day_count'] <= 3:
        return 'TBD'

    # For app_day_count > 3 cases
    if row['app_day_count'] > 3:
        if row['day_missing'] >= row['day_count']:
            return 'missing half of all the days'
        elif row['daily_unlocks'] >= 10 and row['daily_screen_hours'] >= 2:
            return 'working well'
        elif row['daily_unlocks'] >= 10 and row['daily_screen_hours'] < 2:
            return 'not enough screen time'
        elif row['daily_unlocks'] < 10 and row['daily_screen_hours'] >= 2:
            return 'not enough locks'
        elif row['daily_unlocks'] < 10 and row['daily_screen_hours'] < 2:
            return 'neither locks nor screen time is enough'

    return 'TBD'  # Default case if none of the conditions match

# Apply the function to create the note column
df2['note'] = df2.apply(create_note, axis=1)

df2 = df2.sort_values(by=['lapse_last_update','status'])

In [36]:
def time_to_minutes(time_str):
    if time_str == '':
        return float('inf')  # This will put empty strings at the bottom
    try:
        hours, minutes = map(int, time_str.split(':'))
        return hours * 60 + minutes
    except:
        return float('inf')  # Handle any unexpected formats

df2 = df2.sort_values(
    by=['lapse_last_update', 'status'],
    ascending=[True, True],
    key=lambda x: x.map(time_to_minutes) if x.name == 'lapse_last_update' else x
)

In [37]:
# remove all those nan

In [38]:
# Save the files
write_df_to_sheet(df2, participant_manager_ID)

True

In [39]:
# prompt: count rows in df2 where status is 2

# Assuming df2 is defined as in the provided code.
count_working = len(df2[df2['status'] == 2])
count_data_paused = len(df2[df2['status'] == 3])
count_open_slots = 40 - count_working - count_data_paused
# Print the count
print(f"{count_working} is working well")
print(f"{count_data_paused} has data paused")
print(f"Number of open slots: {count_open_slots}")

21 is working well
11 has data paused
Number of open slots: 8


## Screen events data monitor

In [40]:
import datetime
from datetime import timezone, timedelta
import pytz

df = df2.copy()
# Get timezone
ny_tz = pytz.timezone('America/New_York')

# Get today's date
today = datetime.datetime.now(ny_tz)

# Generate the three dates as strings
today_str = today.strftime('%m%d')
yesterday_str = (today - timedelta(days=1)).strftime('%m%d')
day_before_str = (today - timedelta(days=2)).strftime('%m%d')

# Create list of possible column names (both string and int versions)
date_cols = [
    today_str, yesterday_str, day_before_str,  # string versions
    int(today_str), int(yesterday_str), int(day_before_str)  # int versions
]

# Find which version of each date exists in the columns
existing_date_cols = [col for col in date_cols if col in df.columns]

### Data pause drop out notification

if no data is seen in the past 3 days

In [41]:
# Filter rows where all three days are 'data paused'
all_days_paused_mask = df[existing_date_cols].eq('data paused').all(axis=1)
paused_emails = df.loc[all_days_paused_mask, 'Email']

# Convert to comma-separated string
email_list = ', '.join(paused_emails)

print(email_list)

### Data pause notification

If the data is not coming through in the last 12 hours

In [42]:
# Find which version of today's date exists
today_col = next(col for col in [today_str, int(today_str)] if col in df.columns)

# Create masks
today_paused_mask = df[today_col] == 'data paused'

# Get emails where today is paused but not all days are paused
target_emails = df.loc[today_paused_mask & ~all_days_paused_mask, 'Email']

# Convert to comma-separated string
email_list = ', '.join(target_emails)

print(email_list)

nkaganovsky@gmail.com, laurenjrich@outlook.com, frankcampbell495@gmail.com, kimberlysmithy693@gmail.com, jthird253@gmail.com, brownmartins504@gmail.com, girakeira85@gmail.com, mmmoshen@yahoo.com, finplan65@yahoo.com, miller.jennylee@yahoo.com, reevawebb@yahoo.com


### Data resume notification

If yesterday's status is data pause, but today is working

In [43]:
yesterday_col = next(col for col in [yesterday_str, int(yesterday_str)] if col in df.columns)

# Create masks for the conditions
today_started_mask = df[today_col] == 'started'
yesterday_paused_mask = df[yesterday_col] == 'data paused'

# Get emails that meet both conditions
resume_emails = df.loc[today_started_mask & yesterday_paused_mask, 'Email']

# Convert to comma-separated string
email_list = ', '.join(resume_emails)

print(email_list)

## Check the survey filter question

In [44]:
survey_data = pd.read_csv(google_drive_data_path + '/survey_responses.csv')

In [45]:
# prompt: filter by q_13 != 0

# Assuming 'q_13' is a column in your survey_data DataFrame
filtered_survey_data = survey_data[survey_data['q_13'] != 0]
filtered_survey_data['pid']

,pid
2,st012
28,st042


## Generate reminder mail list

Generate a list of emails for certain filter. Copy the output into bcc in gmail

Locate the corresponding email in the document [Emails](https://docs.google.com/document/d/1e5TZJ6ILpku8rVdO_ADUwI-lM8SCIh7xG95jrG73INY/edit?usp=sharing)

In [46]:
# Double check with user_states data

In [47]:
survey_meta_manager = pd.read_csv(google_drive_data_path + '/survey_meta_pivot.csv')

def get_emails_by_day_count(day_count):
    emails = survey_meta_manager[survey_meta_manager['app_day_count'] == day_count]['Email'].tolist()
    if emails:
        return ','.join(emails)
    else:
        return None

In [48]:
survey_meta_manager.head()

,PID,survey1,survey2,survey3,survey4,app_day_count,Email,date_invited
0,ST011,2024-12-03 19:00:00-05:00,NaN,NaN,NaN,8.0,mappscarl@gmail.com,2024-12-04
1,ST012,2024-12-03 19:00:00-05:00,2024-12-10 00:00:00-05:00,NaN,NaN,8.0,djsboca@aol.com,2024-12-04
2,ST013,2024-12-03 19:00:00-05:00,2024-12-10 00:00:00-05:00,NaN,NaN,8.0,santanana2000@yahoo.com,2024-12-04
3,ST014,2024-12-03 19:00:00-05:00,NaN,NaN,NaN,8.0,sadiesantacruz@gmail.com,2024-12-04
4,ST015,2024-12-03 19:00:00-05:00,NaN,NaN,NaN,8.0,jimoreno2055@gmail.com,2024-12-04


### Week 1 completion

In [49]:
# filter those day_count is 7
emails = get_emails_by_day_count(7)
print(emails)

peacepulisic1@gmail.com,owen.micch@gmail.com,alexrhonda37@gmail.com,simmsmba@gmail.com,ssksg73@gmail.com,noahsschool2018@gmail.com


### Week 2 completion

In [50]:
# filter those day_count is 14
emails = get_emails_by_day_count(14)
print(emails)

None


### Week 3 completion

In [51]:
# filter those day_count is 21
emails = get_emails_by_day_count(21)
print(emails)

None


### Daily reminder

In [52]:
# if there is an available survey awaiting
# Create four columns with initial 'unavailable' status
survey_meta_manager['survey1_status'] = 'unavailable'
survey_meta_manager['survey2_status'] = 'unavailable'
survey_meta_manager['survey3_status'] = 'unavailable'
survey_meta_manager['survey4_status'] = 'unavailable'

# Update status to 'complete' if there's a date in the survey columns
# Check if the column exists before accessing it
for i in range(1, 5):
    survey_col = f'survey{i}'
    status_col = f'{survey_col}_status'
    if survey_col in survey_meta_manager.columns:  # Check if column exists
        survey_meta_manager[status_col] = np.where(survey_meta_manager[survey_col].notna(), 'complete', survey_meta_manager[status_col])

# Update status to 'available' if survey is NA and day_count exceeds the threshold
thresholds = [6, 13, 20]
for i, col in enumerate(['survey2', 'survey3', 'survey4'], start=2):
    status_col = f'{col}_status'
    if col in survey_meta_manager.columns:  # Check if column exists
        survey_meta_manager[status_col] = np.where(
            (survey_meta_manager[col].isna()) & (survey_meta_manager['app_day_count'] > thresholds[i-2]),
            'available',
            survey_meta_manager[status_col]
        )

# Save the updated DataFrame
print("The shape of the dataframe is: ", survey_meta_manager.shape)
survey_meta_manager.to_csv(google_drive_data_path + '/survey_meta_manager.csv', index=False)

The shape of the dataframe is:  (49, 12)


In [53]:
survey_meta_manager.head()

,PID,survey1,survey2,survey3,survey4,app_day_count,Email,date_invited,survey1_status,survey2_status,survey3_status,survey4_status
0,ST011,2024-12-03 19:00:00-05:00,NaN,NaN,NaN,8.0,mappscarl@gmail.com,2024-12-04,complete,available,unavailable,unavailable
1,ST012,2024-12-03 19:00:00-05:00,2024-12-10 00:00:00-05:00,NaN,NaN,8.0,djsboca@aol.com,2024-12-04,complete,complete,unavailable,unavailable
2,ST013,2024-12-03 19:00:00-05:00,2024-12-10 00:00:00-05:00,NaN,NaN,8.0,santanana2000@yahoo.com,2024-12-04,complete,complete,unavailable,unavailable
3,ST014,2024-12-03 19:00:00-05:00,NaN,NaN,NaN,8.0,sadiesantacruz@gmail.com,2024-12-04,complete,available,unavailable,unavailable
4,ST015,2024-12-03 19:00:00-05:00,NaN,NaN,NaN,8.0,jimoreno2055@gmail.com,2024-12-04,complete,available,unavailable,unavailable


In [54]:
# Get emails for participants with at least one 'available' survey
target_participants = survey_meta_manager[(survey_meta_manager['survey2_status'] == 'available') |
                                          (survey_meta_manager['survey3_status'] == 'available') |
                                          (survey_meta_manager['survey4_status'] == 'available')]
email_list = target_participants['Email'].tolist()
print(','.join(email_list) or 'None')

mappscarl@gmail.com,sadiesantacruz@gmail.com,jimoreno2055@gmail.com,hongvuh@gmail.com,shelomohtony@gmail.com,brian_60629@yahoo.com,ktippett05@nc.rr.com,notquiterite73@gmail.com,ahaileselassie7459@gmail.com,roald820@gmail.com,peacepulisic1@gmail.com,owen.micch@gmail.com,alexrhonda37@gmail.com,simmsmba@gmail.com,ssksg73@gmail.com,noahsschool2018@gmail.com


### Drop out notification

In [55]:
# If a survey is unanswered for seven days, send out a drop-out notification

In [56]:
# If for three consecutive days, data is not coming through

## Participants from Janel

In [ ]:
sign_up_data = pd.read_excel(google_drive_data_path + '/signup_survey_data_processed_combined.xlsx')
# prompt: filter where sourse == Janel Rana

# Assuming 'sourse' is a column in your sign_up_data DataFrame
filtered_sign_up_data = sign_up_data[sign_up_data['Source'] == 'Janel Rana']

filtered_sign_up_data = filtered_sign_up_data[['Name','Email','Recruit','Note']]
# prompt: In filtered_sign_up_data, create a new column "recruited?" from "Recruit": 1-> yes, 0->'will not', others -> 'TBD'

# Create the 'recruited?' column based on the 'Recruit' column
filtered_sign_up_data['recruited?'] = filtered_sign_up_data['Recruit'].map({
    1: 'yes',
    0: 'will not'
})
filtered_sign_up_data['recruited?'] = filtered_sign_up_data['recruited?'].fillna('TBD')

In [ ]:
Janel_noted_ID = "17oIsLJ4GL5xK5SLQlc2tQ2NGZfuUw4AjWqCy2nC6jd0"
janel_noted_data = get_sheet_as_df(Janel_noted_ID)
participant_manager = pd.read_csv(google_drive_data_path + '/participant_manager.csv')

# prompt: In janel_noted_data and participant_manager, change values in Email all to lower case

# 0. lower case Email in both files
janel_noted_data['Email'] = janel_noted_data['Email'].str.lower()
participant_manager['Email'] = participant_manager['Email'].str.lower()
# Filter filtered_sign_up_data where Email does not exist in janel_noted_data
filtered_df = filtered_sign_up_data[~filtered_sign_up_data['Email'].isin(janel_noted_data['Email'])]

# Get columns present in janel_noted_data but missing in filtered_df
missing_cols = list(set(janel_noted_data.columns) - set(filtered_df.columns))

# Create missing columns in filtered_df and fill with NaN
for col in missing_cols:
    filtered_df[col] = float('nan')

# Now you can safely concatenate
combined_df = pd.concat([janel_noted_data, filtered_df[janel_noted_data.columns]], ignore_index=True)

lasted_status = participant_manager[['Email','1208']]

# prompt: merge lasted_status to combined_df by Email, how = left

combined_df = pd.merge(combined_df, lasted_status, on='Email', how='left')

# prompt: in combined_df, if recuited? = 'yes', update the value in 'note' from '1208'

# Update 'note' column based on 'recruited?' column
combined_df.loc[combined_df['recruited?'] == 'yes', 'note'] = combined_df.loc[combined_df['recruited?'] == 'yes', '1208']

# prompt: drop column '1208'

combined_df = combined_df.drop(columns=['1208'], errors='ignore')

In [ ]:
success = write_df_to_sheet(combined_df, Janel_noted_ID)

Data successfully written to sheet


In [ ]:
def write_df_to_sheet(df, spreadsheet_id):
    """
    Write DataFrame to Google Sheet with handling for special float values
    """
    try:
        # Make a copy to avoid modifying original DataFrame
        df_clean = df.copy()

        # Replace problematic values
        df_clean = df_clean.replace([np.inf, -np.inf], 'Infinity')  # Handle infinity
        df_clean = df_clean.fillna('')  # Replace NaN with empty string

        # Open the spreadsheet
        spreadsheet = gc.open_by_key(spreadsheet_id)
        worksheet = spreadsheet.sheet1

        # Convert DataFrame to list of lists including headers
        values = [df_clean.columns.values.tolist()] + df_clean.values.tolist()

        # Clear existing content and update
        worksheet.clear()
        worksheet.update(values=values, range_name='A1')  # Fixed order of arguments

        print("Data successfully written to sheet")
        return True

    except Exception as e:
        print(f"Error writing to spreadsheet: {e}")
        return False